In [1]:
import pandas as pd
pd.set_option('chained_assignment',None)
import swifter
import numpy as np

In [2]:
y2016_data = pd.read_csv("../Datasets/2016_data.csv")

# The data is already aggregated row-wise, unlike the 2017 data

In [3]:
y2016_data.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Population2010', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI',
       'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev',
       'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI',
       'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev',
       'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI',
       'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev',
       'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI',
       'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev',
       'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI',
       'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev',
       'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI',
       'DIABETES_CrudePrev', 'DIABETES_Crude95CI', 'HIGHCHOL_CrudePrev',
       'HIGHCHOL_Crude95CI', 'KIDNEY_CrudePrev', 'KIDNEY_Crude95CI',
       'LPA_CrudePrev', 'LPA_

In [4]:
keep_columns = ['TractFIPS'] # definitely need to keep the TractFIPS
for column in y2016_data.columns:
    if 'CrudePrev' in column:
        keep_columns.append(column)

In [5]:
y2016_data = y2016_data[keep_columns].copy()

In [6]:
y2016_data.head(3)

,TractFIPS,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,...,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
0,6037206032,28.0,19.9,13.0,31.7,70.0,4.5,8.5,6.9,63.8,...,4.1,33.3,77.6,15.0,29.5,83.7,17.3,38.9,4.0,18.8
1,6037462001,23.3,16.7,16.3,27.1,63.7,3.6,9.4,4.8,62.2,...,3.2,27.0,80.2,15.7,32.0,85.8,15.3,40.2,3.2,18.1
2,6059011504,11.7,8.3,18.6,13.6,49.5,2.0,9.2,2.1,59.3,...,1.6,16.1,81.0,14.1,18.7,78.2,8.5,33.4,1.2,11.9


# Look for measures that are shared between the 2017 and 2016 dataset, to generate trend features

In [7]:
# 2017 measures
y2017_data = pd.read_csv("../Datasets/2017_data.csv")
y2017_measures = set(y2017_data['MeasureId'].unique())

In [8]:
y2017_measures

{'ACCESS2',
 'ARTHRITIS',
 'BINGE',
 'BPHIGH',
 'BPMED',
 'CANCER',
 'CASTHMA',
 'CHD',
 'CHECKUP',
 'CHOLSCREEN',
 'COPD',
 'CSMOKING',
 'DIABETES',
 'HIGHCHOL',
 'KIDNEY',
 'LPA',
 'MHLTH',
 'OBESITY',
 'PHLTH',
 'STROKE'}

# Reformat 2016 data prior to joining with 2017 data
- TractFIPS needs to have leading 0
- Remove measures not part of the 2017 dataset
- Change MEASURE_CrudePrev -> MEASURE_2016


In [9]:
# reformat TractFIPS
y2016_data['TractFIPS'] = y2016_data['TractFIPS'].astype(str).str.pad(width=11, side='left', fillchar='0')

In [10]:
drop_columns = []

for col in y2016_data.columns:
    if col == 'TractFIPS': continue # ignore TractFIPS
    base_measure = col.replace('_CrudePrev', '')
    if base_measure not in y2017_measures:
        drop_columns.append(col)
print('2016 columns to remove')
print(drop_columns)
y2016_data = y2016_data.drop(columns=drop_columns)

2016 columns to remove
['COLON_SCREEN_CrudePrev', 'COREM_CrudePrev', 'COREW_CrudePrev', 'DENTAL_CrudePrev', 'MAMMOUSE_CrudePrev', 'PAPTEST_CrudePrev', 'SLEEP_CrudePrev', 'TEETHLOST_CrudePrev']


In [11]:
# Rename columns
column_remap = {}

for col in y2016_data.columns:
    if col == 'TractFIPS': continue # ignore TractFIPS
    column_remap[col] = col.replace("_CrudePrev", "_2016")
            
y2016_data = y2016_data.rename(columns=column_remap)
y2016_data.head(3)

,TractFIPS,ACCESS2_2016,ARTHRITIS_2016,BINGE_2016,BPHIGH_2016,BPMED_2016,CANCER_2016,CASTHMA_2016,CHD_2016,CHECKUP_2016,...,COPD_2016,CSMOKING_2016,DIABETES_2016,HIGHCHOL_2016,KIDNEY_2016,LPA_2016,MHLTH_2016,OBESITY_2016,PHLTH_2016,STROKE_2016
0,06037206032,28.0,19.9,13.0,31.7,70.0,4.5,8.5,6.9,63.8,...,6.5,15.6,15.5,38.3,4.1,33.3,15.0,29.5,17.3,4.0
1,06037462001,23.3,16.7,16.3,27.1,63.7,3.6,9.4,4.8,62.2,...,6.0,17.8,11.6,31.8,3.2,27.0,15.7,32.0,15.3,3.2
2,06059011504,11.7,8.3,18.6,13.6,49.5,2.0,9.2,2.1,59.3,...,3.4,13.3,4.6,21.1,1.6,16.1,14.1,18.7,8.5,1.2


In [12]:
y2016_data.set_index('TractFIPS', inplace=True)
y2016_data.head(3)

,ACCESS2_2016,ARTHRITIS_2016,BINGE_2016,BPHIGH_2016,BPMED_2016,CANCER_2016,CASTHMA_2016,CHD_2016,CHECKUP_2016,CHOLSCREEN_2016,COPD_2016,CSMOKING_2016,DIABETES_2016,HIGHCHOL_2016,KIDNEY_2016,LPA_2016,MHLTH_2016,OBESITY_2016,PHLTH_2016,STROKE_2016
TractFIPS,,,,,,,,,,,,,,,,,,,,
06037206032,28.0,19.9,13.0,31.7,70.0,4.5,8.5,6.9,63.8,66.0,6.5,15.6,15.5,38.3,4.1,33.3,15.0,29.5,17.3,4.0
06037462001,23.3,16.7,16.3,27.1,63.7,3.6,9.4,4.8,62.2,65.7,6.0,17.8,11.6,31.8,3.2,27.0,15.7,32.0,15.3,3.2
06059011504,11.7,8.3,18.6,13.6,49.5,2.0,9.2,2.1,59.3,54.5,3.4,13.3,4.6,21.1,1.6,16.1,14.1,18.7,8.5,1.2


# Join to the current 2017 dataset, and add the trend info

In [13]:
# need to make sure these columns get read as strings
fips_columns = [
    'CountyFIPS',
    'CityFIPS',
    'TractFIPS'
]
dtypes_map = {x: 'str' for x in fips_columns}
training_data = pd.read_csv('../Datasets/training_data.csv', dtype=dtypes_map)

In [14]:
joined_data = training_data.join(y2016_data, on='TractFIPS')

In [15]:
measure_columns = []

for measure in y2017_measures:
    measure_trend = f'{measure}_trend' # new column
    measure_2017 = f'{measure}_2017'
    measure_2016 = f'{measure}_2016'
    
    joined_data[measure_trend] = (joined_data[measure_2017] - joined_data[measure_2016]) / joined_data[measure_2016]
    
    measure_columns.append(measure_2017)
    measure_columns.append(measure_trend)
    
    joined_data.drop(columns=[measure_2016], inplace=True)

In [16]:
final_col_order = [
    'StateAbbr',
    'StateDesc',
    'CityName',
    'CountyFIPS',
    'TractFIPS',
    'Population_2017'
] + measure_columns

In [17]:
joined_data = joined_data[final_col_order].copy()

In [18]:
joined_data.head(3)

,StateAbbr,StateDesc,CityName,CountyFIPS,TractFIPS,Population_2017,CHECKUP_2017,CHECKUP_trend,KIDNEY_2017,KIDNEY_trend,...,CASTHMA_2017,CASTHMA_trend,ACCESS2_2017,ACCESS2_trend,CANCER_2017,CANCER_trend,CHOLSCREEN_2017,CHOLSCREEN_trend,DIABETES_2017,DIABETES_trend
0,AL,Alabama,Birmingham,01073,01073000100,3042,74.8,-0.023499,4.3,0.023810,...,13.4,0.135593,24.4,0.047210,5.3,0.019231,78.6,0.087137,17.4,-0.049180
1,AL,Alabama,Birmingham,01073,01073000300,2735,72.1,-0.024357,4.6,0.069767,...,12.5,0.136364,30.4,0.066667,4.7,0.021739,76.0,0.088825,19.8,0.005076
2,AL,Alabama,Birmingham,01073,01073000400,3338,76.6,-0.019206,4.2,0.000000,...,13.3,0.127119,24.7,0.033473,5.3,0.019231,80.2,0.113889,18.4,-0.061224


# Join in the 2020, to calculate diabetes trend between 2017-2020

In [19]:
y2020_data = pd.read_csv("../Datasets/2020_data.csv")

In [20]:
# this is just a map of TractFIPS to 2020 diabetes rates

diabetes_data_2020 = y2020_data.query('MeasureId == "DIABETES"')
diabetes_data_2020['LocationID'] = diabetes_data_2020['LocationID'].astype(str).str.pad(width=11, side='left', fillchar='0')

diabetes_rate_map = dict(zip(
    diabetes_data_2020['LocationID'], # synonymous with TractFIPS
    diabetes_data_2020['Data_Value']
))

[print(v) for i, v in enumerate(diabetes_rate_map.items()) if i < 3];

('01055000300', 24.2)
('01073000100', 19.1)
('01073000500', 23.7)


In [21]:
joined_data['DIABETES_2020'] = joined_data['TractFIPS'].swifter.apply(diabetes_rate_map.get)

/home/tychi/.local/lib/python3.8/site-packages/swifter/swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/27120 [00:00<?, ?it/s]

In [22]:
joined_data.head(3)

,StateAbbr,StateDesc,CityName,CountyFIPS,TractFIPS,Population_2017,CHECKUP_2017,CHECKUP_trend,KIDNEY_2017,KIDNEY_trend,...,CASTHMA_trend,ACCESS2_2017,ACCESS2_trend,CANCER_2017,CANCER_trend,CHOLSCREEN_2017,CHOLSCREEN_trend,DIABETES_2017,DIABETES_trend,DIABETES_2020
0,AL,Alabama,Birmingham,01073,01073000100,3042,74.8,-0.023499,4.3,0.023810,...,0.135593,24.4,0.047210,5.3,0.019231,78.6,0.087137,17.4,-0.049180,19.1
1,AL,Alabama,Birmingham,01073,01073000300,2735,72.1,-0.024357,4.6,0.069767,...,0.136364,30.4,0.066667,4.7,0.021739,76.0,0.088825,19.8,0.005076,21.8
2,AL,Alabama,Birmingham,01073,01073000400,3338,76.6,-0.019206,4.2,0.000000,...,0.127119,24.7,0.033473,5.3,0.019231,80.2,0.113889,18.4,-0.061224,18.9


# Finally add the diabetes_change_rate

IE if a census tract in 2017 had 10% diabetes rate, and then had 15% diabetes rate in 2020\
then the diabetes_change_rate would be 0.5

In [23]:
joined_data['diabetes_change_rate'] =\
    (joined_data['DIABETES_2020'] - joined_data['DIABETES_2017']) / joined_data['DIABETES_2017']

In [24]:
joined_data.columns

Index(['StateAbbr', 'StateDesc', 'CityName', 'CountyFIPS', 'TractFIPS',
       'Population_2017', 'CHECKUP_2017', 'CHECKUP_trend', 'KIDNEY_2017',
       'KIDNEY_trend', 'PHLTH_2017', 'PHLTH_trend', 'LPA_2017', 'LPA_trend',
       'BPHIGH_2017', 'BPHIGH_trend', 'BINGE_2017', 'BINGE_trend',
       'OBESITY_2017', 'OBESITY_trend', 'HIGHCHOL_2017', 'HIGHCHOL_trend',
       'ARTHRITIS_2017', 'ARTHRITIS_trend', 'STROKE_2017', 'STROKE_trend',
       'CSMOKING_2017', 'CSMOKING_trend', 'CHD_2017', 'CHD_trend',
       'MHLTH_2017', 'MHLTH_trend', 'BPMED_2017', 'BPMED_trend', 'COPD_2017',
       'COPD_trend', 'CASTHMA_2017', 'CASTHMA_trend', 'ACCESS2_2017',
       'ACCESS2_trend', 'CANCER_2017', 'CANCER_trend', 'CHOLSCREEN_2017',
       'CHOLSCREEN_trend', 'DIABETES_2017', 'DIABETES_trend', 'DIABETES_2020',
       'diabetes_change_rate'],
      dtype='object')

In [29]:
print(joined_data.isna().sum().sum())
joined_data = joined_data.dropna()

1


In [30]:
joined_data

,StateAbbr,StateDesc,CityName,CountyFIPS,TractFIPS,Population_2017,CHECKUP_2017,CHECKUP_trend,KIDNEY_2017,KIDNEY_trend,...,ACCESS2_2017,ACCESS2_trend,CANCER_2017,CANCER_trend,CHOLSCREEN_2017,CHOLSCREEN_trend,DIABETES_2017,DIABETES_trend,DIABETES_2020,diabetes_change_rate
0,AL,Alabama,Birmingham,01073,01073000100,3042,74.8,-0.023499,4.3,0.023810,...,24.4,0.047210,5.3,0.019231,78.6,0.087137,17.4,-0.049180,19.1,0.097701
1,AL,Alabama,Birmingham,01073,01073000300,2735,72.1,-0.024357,4.6,0.069767,...,30.4,0.066667,4.7,0.021739,76.0,0.088825,19.8,0.005076,21.8,0.101010
2,AL,Alabama,Birmingham,01073,01073000400,3338,76.6,-0.019206,4.2,0.000000,...,24.7,0.033473,5.3,0.019231,80.2,0.113889,18.4,-0.061224,18.9,0.027174
3,AL,Alabama,Birmingham,01073,01073000500,2864,78.2,-0.016352,5.3,0.019231,...,25.2,-0.023256,6.0,0.034483,80.5,0.119611,22.4,-0.062762,23.7,0.058036
4,AL,Alabama,Birmingham,01073,01073000700,2577,78.1,-0.017610,6.4,0.032258,...,29.0,-0.013605,5.9,0.035088,77.4,0.125000,25.0,-0.042146,24.7,-0.012000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26873,WY,Wyoming,Cheyenne,56021,56021001401,3961,64.4,0.014173,2.5,0.086957,...,14.7,0.088889,6.3,0.032787,76.0,0.013333,8.1,0.025316,7.8,-0.037037
26874,WY,Wyoming,Cheyenne,56021,56021001402,1913,68.9,0.013235,3.1,0.068966,...,13.6,0.079365,9.0,0.034483,78.6,0.030144,8.8,0.011494,9.4,0.068182
26875,WY,Wyoming,Cheyenne,56021,56021001501,3312,64.7,0.014107,2.3,0.045455,...,14.5,0.098485,6.4,0.032258,76.5,0.022727,7.4,0.000000,7.7,0.040541
26876,WY,Wyoming,Cheyenne,56021,56021001502,4518,65.1,0.023585,3.0,-0.032258,...,16.6,-0.034884,6.9,0.029851,74.5,0.043417,9.4,-0.078431,8.9,-0.053191
